In [1]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

import scipy.signal
import holidays

import tensorflow as tf

from pandas.tseries.offsets import MonthEnd

import seaborn as sb

import IPython

In [2]:
# Aux Functions for Solcast and PV data processing

def get_solcast(path):
    file = pd.read_csv(path)
    file.index = pd.to_datetime(file['PeriodEnd'])
    file = file.resample('15T').pad()
    
    return file


def get_pv(path):
    file = pd.read_csv(path)
    file.index = pd.to_datetime(file['datetime_utc'])
    file = file.resample('15T').mean()
    
    return file


def get_solcastPV(df1, df2):
    '''
    df1: PV dataframe
    df2: Solcast dataframe
    '''
    
    # Filter both dataframes for 2019 and 2020
    try:
        temp_df1 = df1['2019':'2021-04-01']
        temp_df2 = df2['2019':'2021-04-01']
        
        # Check if data is complete. If not, match the smaller indexes
        if temp_df2.shape[0] < temp_df1.shape[0]:
            last_entry = temp_df2.index
            temp_df1 = temp_df1['2019':'{}'.format(temp_df2.index[-1].tz_convert(None))]


        # Only considering 2019 and 2020 since data is complete for that period
        temp_data = pd.DataFrame({'PV': temp_df1['pv'].values}, index=temp_df1.index)
        for i in np.arange(3, len(temp_df2.columns)):
            temp_data[temp_df2.columns[i]] = temp_df2[temp_df2.columns[i]].shift(-1).values
            
        return temp_data
    except:
        temp_df1 = df1['2019':'2020']
        temp_df2 = df2['2019':'2020']
        
        # Check if data is complete. If not, match the smaller indexes
        if temp_df2.shape[0] < temp_df1.shape[0]:
            last_entry = temp_df2.index
            temp_df1 = temp_df1['2019':'{}'.format(temp_df2.index[-1].tz_convert(None))]


        # Only considering 2019 and 2020 since data is complete for that period
        temp_data = pd.DataFrame({'PV': temp_df1['pv'].values}, index=temp_df1.index)
        for i in np.arange(3, len(temp_df2.columns)):
            temp_data[temp_df2.columns[i]] = temp_df2[temp_df2.columns[i]].shift(-1).values
            
        return temp_data

In [15]:
# Get data and build a dictionary for preprocessing

data = {}

folders = glob.glob('C:/Users/FEEL/Jupyter/ecgomes/upacs_study/data/*')
for folder in folders:
    # Load each of the files inside the folder
    temp_pv = get_pv('{}/pv.csv'.format(folder))
    temp_solcast = get_solcast('{}/solcast.csv'.format(folder))
    
    # Join the files into a single dataframe
    temp_upac = get_solcastPV(temp_pv, temp_solcast)
    
    temp_name = folder.split('\\')[1]
    data[temp_name] = temp_upac
    
    print('{} date range: {} - {}'.format(temp_name, temp_upac.index[0], temp_upac.index[-1]))

upac02 date range: 2019-01-01 00:00:00 - 2020-06-15 22:30:00
upac06 date range: 2019-01-01 00:00:00 - 2020-06-15 22:30:00
upac08 date range: 2019-01-01 00:00:00 - 2021-04-01 23:45:00
upac09 date range: 2019-01-01 00:00:00 - 2020-12-31 23:45:00
upac13 date range: 2019-01-01 00:00:00 - 2020-12-31 23:45:00


In [16]:
# Aux Functions for adding 2D time information

import datetime

def days_2d(df):
    '''
    Adds 2D time information for single days
    df: dataframe to add the information
    '''
    # Map the index into seconds
    timestamp_s = pd.to_datetime(df.index.values).map(datetime.datetime.timestamp)
    
    # Since we're calculating the cos and sin values from seconds, it's 60 seconds into 60 min into 24 hours per day
    day_calc = 24*60*60
    
    # Calculate the values
    dayx = np.cos((2*np.pi/day_calc) * timestamp_s)
    dayy = np.sin((2*np.pi/day_calc) * timestamp_s)
    
    return dayx, dayy
    

def years_2d(df):
    '''
    Adds 2D time representation throught a year
    df: dataframe to add the information
    '''
    # Add Year Information

    day_year = df.index.dayofyear
    year_constant = 365.2524

    yearx = np.cos((2*np.pi/year_constant) * day_year)
    yeary = np.sin((2*np.pi/year_constant) * day_year)
    
    return yearx, yeary

In [17]:
# Add the 2D time information to the data

for upac in data.keys():
    dayx, dayy = days_2d(data[upac])
    yearx, yeary = years_2d(data[upac])
    
    data[upac]['Day X'] = dayx
    data[upac]['Day Y'] = dayy
    
    data[upac]['Year X'] = yearx
    data[upac]['Year Y'] = yeary

In [18]:
# Split the data for training, validation and testing

data_train = {}
data_val = {}
data_test = {}

for upac in data.keys():
    data_train[upac] = data[upac]['2019']
    data_val[upac] = data[upac]['2020-01':'2020-03']
    data_test[upac] = data[upac]['2020-04':]

C:\Users\FEEL\AppData\Local\Temp/ipykernel_9884/2785044146.py:8: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  data_train[upac] = data[upac]['2019']


In [19]:
# Aux Function for filtering data

def filter_by_points(df, frequency='D', num_points=1440, return_dictionary=False):
    
    df_dropped = df.dropna()
    grouper = df_dropped.groupby(pd.Grouper(freq=frequency))
    
    output = 0
    if return_dictionary:
        new_dict = {}
        for i in grouper:
            if (len(i[1]) != num_points):
                pass
            else:
                new_dict[i[0]] = pd.DataFrame(i[1])
        output = new_dict
    else:
        new_df = pd.DataFrame({})
        for i in grouper:
            if (len(i[1]) != num_points):
                pass
            else:
                new_df = new_df.append(pd.DataFrame(i[1]))
        output = new_df
            
    return output

In [20]:
# Filter the data by number of points that should be present in a single day

filtered_train = {}
filtered_val = {}
filtered_test = {}

for upac in data_train.keys():
    filtered_train[upac] = filter_by_points(data_train[upac], frequency='D', num_points=1440/15)
    filtered_val[upac] = filter_by_points(data_val[upac], frequency='D', num_points=1440/15)
    filtered_test[upac] = filter_by_points(data_test[upac], frequency='D', num_points=1440/15)

In [21]:
# Select columns to use

USED_COLUMNS = ['PV', 
                'AirTemp', 
                'CloudOpacity',
                'Ghi',
                'GtiFixedTilt', 
                'Day Y', 'Day X',
                'Year Y', 'Year X']

In [22]:
# Data Normalization
# We don't want to normalize PV so we can capture diferences more easily

# Feature range
# PV - greater than 0
# AirTemp - Unchanged
# Cloud Opacity and Ghi - between 0 and 1
# Day X, Y and Year X and Y - already between -1 and 1

MAX_OPACITY = 100
MAX_GHI = 1023
MAX_GTI = 1071

normalized_train = {}
normalized_val = {}
normalized_test = {}

for upac in filtered_train.keys():
    normalized_train[upac] = filtered_train[upac][USED_COLUMNS].copy(deep=True)
    normalized_val[upac] = filtered_val[upac][USED_COLUMNS].copy(deep=True)
    normalized_test[upac] = filtered_test[upac][USED_COLUMNS].copy(deep=True)
    
    normalized_train[upac]['CloudOpacity'] = normalized_train[upac]['CloudOpacity'] / MAX_OPACITY
    normalized_val[upac]['CloudOpacity'] = normalized_val[upac]['CloudOpacity'] / MAX_OPACITY
    normalized_test[upac]['CloudOpacity'] = normalized_test[upac]['CloudOpacity'] / MAX_OPACITY
    
    normalized_train[upac]['Ghi'] = normalized_train[upac]['Ghi'] / MAX_GHI
    normalized_val[upac]['Ghi'] = normalized_val[upac]['Ghi'] / MAX_GHI
    normalized_test[upac]['Ghi'] = normalized_test[upac]['Ghi'] / MAX_GHI
    
    normalized_train[upac]['GtiFixedTilt'] = normalized_train[upac]['GtiFixedTilt'] / MAX_GTI
    normalized_val[upac]['GtiFixedTilt'] = normalized_val[upac]['GtiFixedTilt'] / MAX_GTI
    normalized_test[upac]['GtiFixedTilt'] = normalized_test[upac]['GtiFixedTilt'] / MAX_GTI

In [25]:
# Aux Function DNN Specific - Data Reshaping and Windowing

def split_sequence(sequence, n_steps, n_intervals):
    X = list()
    for i in np.arange(0, len(sequence), n_intervals):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x = sequence[i:end_ix]
        X.append(seq_x)
    return np.array(X)

def split_dataframe(df, input_width):
    current_list = []
    for i in df.columns[1:]:
        current_sequence = split_sequence(df[i], input_width, 1)
        current_list.append(current_sequence)
        
    stacked_list = np.stack(current_list, axis=-1)
    
    return stacked_list


def do_windowing(df, input_width=1, y_col='PV'):
    '''
    Perform the windowing on the dataframe
    df: input dataframe to perform windowing
    input_width: temporal dimension size or number of timesteps
    y_col: name of column for the prediction
    '''
    
    temp_x = split_dataframe(df, input_width)
    temp_y = df[y_col].values[input_width-1:-1]
    
    return temp_x, temp_y

In [26]:
# Do windowing on the data

X_train = {}
y_train = {}

X_val = {}
y_val = {}

X_test = {}
y_test = {}

for upac in normalized_train.keys():
    trainx = normalized_train[upac].drop('PV', axis=1)
    trainy = normalized_train[upac]['PV']
    valx = normalized_val[upac].drop('PV', axis=1)
    valy = normalized_val[upac]['PV']
    testx = normalized_test[upac].drop('PV', axis=1)
    testy = normalized_test[upac]['PV']
    
    X_train[upac] = trainx
    X_val[upac] = valx
    X_test[upac] = testx
    
    y_train[upac] = trainy
    y_val[upac] = valy
    y_test[upac] = testy

In [27]:
# Aux Functions to compile the model and add necessary info

def compile_and_fit(model, train_x, train_y, val_x, val_y,
                    max_epochs=1000, es_patience=200, rlr_patience=100, 
                    adam_lr=0.001):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=es_patience,
                                                      mode='min')
    
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                     patience=rlr_patience)

    model.compile(loss=tf.keras.losses.mean_squared_error,
                  optimizer=tf.optimizers.Adam(learning_rate=adam_lr),
                  metrics=[tf.metrics.MeanAbsoluteError(name='mae')])

    history = model.fit(train_x, train_y,
                        batch_size=32,
                        epochs=max_epochs,
                        validation_data=(val_x, val_y),
                        callbacks=[early_stopping, reduce_lr])
    return history

In [37]:
# Aux Function for predicting and storing values

def do_predictions(dictionary, save_path, X, y, index):
    # Go through each model in the dictionary
    for model in dictionary.keys():
        print('Doing {}'.format(model))
        
        temp_path = '{}/{}.csv'.format(save_path, model)
        
        y_pred = dictionary[model].predict(X, verbose=1)
        y_pred = pd.DataFrame(y_pred[:, :], columns=['PV'],
                              index=index)
        
        y_pred.to_csv(temp_path)
        
    # Also save ground-truth data at the end of the loop
    y_true = pd.DataFrame(y, columns=['PV'],
                          index=index)
    
    temp_path_gt = '{}/gt.csv'.format(save_path)
    y_true.to_csv(temp_path_gt)
    
    
def predict_upacs(model_dictionary, upac_name, X_train, y_train, X_val, y_val, X_test, y_test):
    # Simply call the function above for each of the settings to simplify
    
    print('Doing training for {}'.format(upac_name))
    temp_path = 'results/tabnet/{}_ghi+gti/train'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_train[upac_name], 
                   y=y_train[upac_name],
                   index=normalized_train[upac_name].index)
    IPython.display.clear_output()
    
    print('Doing validation for {}'.format(upac_name))
    temp_path = 'results/tabnet/{}_ghi+gti/val'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_val[upac_name], 
                   y=y_val[upac_name],
                   index=normalized_val[upac_name].index)
    IPython.display.clear_output()
    
    print('Doing testing for {}'.format(upac_name))
    temp_path = 'results/tabnet/{}_ghi+gti/test'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_test[upac_name], 
                   y=y_test[upac_name],
                   index=normalized_test[upac_name].index)
    IPython.display.clear_output()

In [28]:
# Import the TabNet implementation

# From: https://github.com/titu1994/tf-TabNet
from tabnet import StackedTabNetRegressor

In [29]:
# Optuna for TabNet hyperparameter optimization

import optuna
import sklearn

import random

random.seed(69)
np.random.seed(69)
tf.random.set_seed(69)

def optuna_create_model(trial):
    # Do search for n_estimators, max_depth, reg_alpha and reg_lambda
    #sug_layers = trial.suggest_int('num_layers', 1, 4)
    sug_fdim = trial.suggest_int('feature_dim', 32, 72)
    sug_odim = trial.suggest_int('output_dim', 4, 12)
    sug_dsteps = trial.suggest_int('num_decision_steps', 1, 4)
    
    sug_model = StackedTabNetRegressor(feature_columns=None,
                                       num_layers=1, #sug_layers,
                                       num_regressors=1, 
                                       feature_dim=sug_fdim,
                                       num_features=8, 
                                       output_dim=sug_odim,
                                       num_decision_steps=sug_dsteps,
                                       num_groups=1)
    
    return sug_model


def optuna_create_training(model):
    compile_and_fit(model, X_train['upac08'], y_train['upac08'],
                    X_val['upac08'], y_val['upac08'],
                    max_epochs=50, rlr_patience=10, es_patience=20)
    
    
def optuna_create_evaluation(model):
    temp_yhat = model.predict(X_val['upac08'])
    return sklearn.metrics.mean_squared_error(y_val['upac08'], temp_yhat)
    
    
def objective(trial):
    # Instantiate the model
    temp_model = optuna_create_model(trial)
    
    # Train the model
    optuna_create_training(temp_model)
    
    # Evaluate model
    metrics_val = optuna_create_evaluation(temp_model)
    
    return metrics_val

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2022-07-28 17:25:53,444] A new study created in memory with name: no-name-27e8e2e1-11b9-4e22-a84f-9c9608c59f9b
c:\users\feel\jupyter\ecgomes\upacs_study\venv\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1/50
1089/1089 [==============================] - 3s 1ms/step - loss: 1644220.8750 - mae: 712.5095 - val_loss: 1243517.1250 - val_mae: 571.9921 - lr: 0.0010
Epoch 2/50
1089/1089 [==============================] - 1s 1ms/step - loss: 1270171.1250 - mae: 588.8021 - val_loss: 869593.1875 - val_mae: 448.4728 - lr: 0.0010
Epoch 3/50
1089/1089 [==============================] - 1s 1ms/step - loss: 846714.3750 - mae: 457.0482 - val_loss: 545072.2500 - val_mae: 345.8116 - lr: 0.0010
Epoch 4/50
1089/1089 [==============================] - 1s 1ms/step - loss: 520005.1562 - mae: 349.7375 - val_loss: 334833.6562 - val_mae: 275.8424 - lr: 0.0010
Epoch 5/50
1089/1089 [==============================] - 1s 1ms/step - loss: 314046.1562 - mae: 275.6707 - val_loss: 220491.6094 - val_mae: 234.6279 - lr: 0.0010
Epoch 6/50
1089/1089 [==============================] - 1s 1ms/step - loss: 201929.0312 - mae: 226.8295 - val_loss: 171013.5469 - val_mae: 203.2840 - lr: 0.0010
Epoch 7/50
1089/1089 [=========

1089/1089 [==============================] - 6s 3ms/step - loss: 1539332.6250 - mae: 709.8397 - val_loss: 1071315.5000 - val_mae: 590.7015 - lr: 0.0010
Epoch 2/50
1089/1089 [==============================] - 3s 3ms/step - loss: 1017943.6875 - mae: 603.5064 - val_loss: 652014.4375 - val_mae: 487.1116 - lr: 0.0010
Epoch 3/50
1089/1089 [==============================] - 3s 3ms/step - loss: 633206.5625 - mae: 506.7497 - val_loss: 412604.8438 - val_mae: 408.3971 - lr: 0.0010
Epoch 4/50
1089/1089 [==============================] - 3s 3ms/step - loss: 423297.4062 - mae: 422.4138 - val_loss: 275590.8438 - val_mae: 332.1918 - lr: 0.0010
Epoch 5/50
1089/1089 [==============================] - 3s 3ms/step - loss: 318977.4375 - mae: 357.9543 - val_loss: 266444.7500 - val_mae: 301.8371 - lr: 0.0010
Epoch 6/50
1089/1089 [==============================] - 3s 3ms/step - loss: 264648.6562 - mae: 313.6446 - val_loss: 189899.2812 - val_mae: 256.7070 - lr: 0.0010
Epoch 7/50
1089/1089 [====================

1089/1089 [==============================] - 1s 961us/step - loss: 177707.5781 - mae: 211.6880 - val_loss: 164371.8906 - val_mae: 195.0402 - lr: 0.0010
Epoch 11/50
1089/1089 [==============================] - 1s 1ms/step - loss: 146464.0781 - mae: 193.3767 - val_loss: 141752.0625 - val_mae: 183.3438 - lr: 0.0010
Epoch 12/50
1089/1089 [==============================] - 1s 1ms/step - loss: 130319.1797 - mae: 183.2485 - val_loss: 141103.8594 - val_mae: 180.3042 - lr: 0.0010
Epoch 13/50
1089/1089 [==============================] - 1s 1ms/step - loss: 120939.0312 - mae: 176.2142 - val_loss: 131289.8281 - val_mae: 171.2925 - lr: 0.0010
Epoch 14/50
1089/1089 [==============================] - 1s 1ms/step - loss: 115419.4531 - mae: 170.4010 - val_loss: 140682.7344 - val_mae: 178.7161 - lr: 0.0010
Epoch 15/50
1089/1089 [==============================] - 1s 1ms/step - loss: 111087.3203 - mae: 165.2586 - val_loss: 118843.2734 - val_mae: 165.1264 - lr: 0.0010
Epoch 16/50
1089/1089 [===============

...

1089/1089 [==============================] - 1s 1ms/step - loss: 101334.1016 - mae: 150.8113 - val_loss: 117390.6953 - val_mae: 155.1466 - lr: 0.0010
Epoch 34/50
1089/1089 [==============================] - 1s 1ms/step - loss: 101344.1328 - mae: 149.8521 - val_loss: 118235.3359 - val_mae: 155.3277 - lr: 0.0010
Epoch 35/50
1089/1089 [==============================] - 1s 1ms/step - loss: 101080.2969 - mae: 149.3747 - val_loss: 115674.1641 - val_mae: 155.7811 - lr: 0.0010
Epoch 36/50
1089/1089 [==============================] - 1s 1ms/step - loss: 100833.5000 - mae: 149.0844 - val_loss: 116613.8281 - val_mae: 155.2500 - lr: 0.0010
Epoch 37/50
1089/1089 [==============================] - 1s 1ms/step - loss: 100740.5625 - mae: 149.2563 - val_loss: 119233.0312 - val_mae: 157.4014 - lr: 0.0010
Epoch 38/50
1089/1089 [==============================] - 1s 1ms/step - loss: 100579.5938 - mae: 149.3537 - val_loss: 117783.4141 - val_mae: 156.4431 - lr: 0.0010
Epoch 39/50
1089/1089 [=================

1089/1089 [==============================] - 1s 1ms/step - loss: 100905.8438 - mae: 149.7062 - val_loss: 121656.1016 - val_mae: 156.6035 - lr: 0.0010
Epoch 33/50
1089/1089 [==============================] - 1s 1ms/step - loss: 100672.6562 - mae: 149.8214 - val_loss: 115030.8984 - val_mae: 151.8534 - lr: 0.0010
Epoch 34/50
1089/1089 [==============================] - 1s 1ms/step - loss: 100731.8281 - mae: 148.9475 - val_loss: 115870.2109 - val_mae: 153.5320 - lr: 0.0010
Epoch 35/50
1089/1089 [==============================] - 1s 1ms/step - loss: 100588.9688 - mae: 149.0378 - val_loss: 112539.2578 - val_mae: 151.0197 - lr: 0.0010
Epoch 36/50
1089/1089 [==============================] - 1s 1ms/step - loss: 100177.9688 - mae: 148.5215 - val_loss: 113040.5938 - val_mae: 151.0023 - lr: 0.0010
Epoch 37/50
1089/1089 [==============================] - 1s 1ms/step - loss: 100259.8750 - mae: 149.0321 - val_loss: 115586.9375 - val_mae: 151.6237 - lr: 0.0010
Epoch 38/50
1089/1089 [=================

1089/1089 [==============================] - 1s 953us/step - loss: 103109.2109 - mae: 152.6537 - val_loss: 122287.9844 - val_mae: 159.6537 - lr: 0.0010
Epoch 32/50
1089/1089 [==============================] - 1s 950us/step - loss: 102527.1484 - mae: 152.2000 - val_loss: 125057.0703 - val_mae: 160.7544 - lr: 0.0010
Epoch 33/50
1089/1089 [==============================] - 1s 963us/step - loss: 102542.8438 - mae: 152.3172 - val_loss: 119137.9219 - val_mae: 154.9979 - lr: 0.0010
Epoch 34/50
1089/1089 [==============================] - 1s 958us/step - loss: 102312.9531 - mae: 151.3183 - val_loss: 118229.7266 - val_mae: 155.4985 - lr: 0.0010
Epoch 35/50
1089/1089 [==============================] - 1s 958us/step - loss: 102293.0234 - mae: 151.3698 - val_loss: 115868.8203 - val_mae: 153.7023 - lr: 0.0010
Epoch 36/50
1089/1089 [==============================] - 1s 954us/step - loss: 102032.2031 - mae: 150.8410 - val_loss: 120080.5469 - val_mae: 158.1883 - lr: 0.0010
Epoch 37/50
1089/1089 [=====

1089/1089 [==============================] - 1s 1ms/step - loss: 100490.0781 - mae: 148.6800 - val_loss: 122021.7500 - val_mae: 159.9740 - lr: 0.0010
Epoch 31/50
1089/1089 [==============================] - 1s 1ms/step - loss: 100740.9062 - mae: 148.8645 - val_loss: 119193.2891 - val_mae: 155.9403 - lr: 0.0010
Epoch 32/50
1089/1089 [==============================] - 1s 1ms/step - loss: 100170.2734 - mae: 148.2194 - val_loss: 120727.0000 - val_mae: 157.0236 - lr: 0.0010
Epoch 33/50
1089/1089 [==============================] - 1s 1ms/step - loss: 99980.1719 - mae: 148.7347 - val_loss: 114039.5938 - val_mae: 149.9245 - lr: 0.0010
Epoch 34/50
1089/1089 [==============================] - 1s 1ms/step - loss: 99814.1641 - mae: 147.4725 - val_loss: 116559.5703 - val_mae: 151.7974 - lr: 0.0010
Epoch 35/50
1089/1089 [==============================] - 1s 1ms/step - loss: 99866.5703 - mae: 147.5862 - val_loss: 113200.3203 - val_mae: 151.1515 - lr: 0.0010
Epoch 36/50
1089/1089 [====================

1089/1089 [==============================] - 1s 1ms/step - loss: 99287.2812 - mae: 148.5709 - val_loss: 116820.9531 - val_mae: 155.5701 - lr: 1.0000e-04
Epoch 30/50
1089/1089 [==============================] - 1s 1ms/step - loss: 99095.7578 - mae: 148.5998 - val_loss: 119165.6250 - val_mae: 157.4087 - lr: 1.0000e-04
Epoch 31/50
1089/1089 [==============================] - 1s 1ms/step - loss: 99143.3359 - mae: 148.3886 - val_loss: 120290.3828 - val_mae: 157.7287 - lr: 1.0000e-04
Epoch 32/50
1089/1089 [==============================] - 1s 1ms/step - loss: 99129.8047 - mae: 148.4542 - val_loss: 120646.4453 - val_mae: 158.1126 - lr: 1.0000e-04
Epoch 33/50
1089/1089 [==============================] - 1s 1ms/step - loss: 99052.2656 - mae: 148.3175 - val_loss: 119255.7188 - val_mae: 156.0043 - lr: 1.0000e-04
Epoch 34/50
1089/1089 [==============================] - 1s 1ms/step - loss: 99038.7891 - mae: 148.1077 - val_loss: 119539.9453 - val_mae: 157.1401 - lr: 1.0000e-04
Epoch 35/50
1089/1089 

In [30]:
# TabNet best params (all fetures)

study.best_params

{'feature_dim': 39, 'output_dim': 11, 'num_decision_steps': 1}

In [36]:
# Train 10 Models for UPAC08 - Top k

N_MODELS = 10
upac08_GhiGti = {}

for i in np.arange(N_MODELS):
    IPython.display.clear_output()
    print('Current Model: {:02d}'.format(i+1))
    
    tabnet_regressor = StackedTabNetRegressor(feature_columns=None,
                                              num_layers=1, #study.best_params['num_layers'],
                                              num_regressors=1, 
                                              feature_dim=study.best_params['feature_dim'],
                                              num_features=4, 
                                              output_dim=study.best_params['output_dim'],
                                              num_decision_steps=study.best_params['num_decision_steps'],
                                              num_groups=1)
    current_net_history = compile_and_fit(tabnet_regressor, 
                                          X_train['upac08'], y_train['upac08'],
                                          X_val['upac08'], y_val['upac08'])
    
    tabnet_regressor.save('models/tabnet/upac08_ghi+gti/Model {:02d}'.format(i+1), save_format='tf')
    upac08_GhiGti['Model {:02d}'.format(i+1)] = tabnet_regressor

Current Model: 10
Epoch 1/1000
1089/1089 [==============================] - 2s 1ms/step - loss: 1678902.2500 - mae: 718.8348 - val_loss: 1318731.3750 - val_mae: 596.4104 - lr: 0.0010
Epoch 2/1000
1089/1089 [==============================] - 1s 974us/step - loss: 1420019.3750 - mae: 634.3806 - val_loss: 1051325.6250 - val_mae: 516.3365 - lr: 0.0010
Epoch 3/1000
1089/1089 [==============================] - 1s 974us/step - loss: 1092781.1250 - mae: 534.1730 - val_loss: 768095.8750 - val_mae: 420.9233 - lr: 0.0010
Epoch 4/1000
1089/1089 [==============================] - 1s 964us/step - loss: 784612.1875 - mae: 437.6797 - val_loss: 534634.0625 - val_mae: 350.9758 - lr: 0.0010
Epoch 5/1000
1089/1089 [==============================] - 1s 971us/step - loss: 536865.1875 - mae: 357.1194 - val_loss: 365120.2188 - val_mae: 287.3880 - lr: 0.0010
Epoch 6/1000
1089/1089 [==============================] - 1s 977us/step - loss: 358285.3438 - mae: 294.0498 - val_loss: 256781.6875 - val_mae: 247.8892 - 

1089/1089 [==============================] - 1s 983us/step - loss: 99649.2422 - mae: 148.7020 - val_loss: 115472.3594 - val_mae: 153.4647 - lr: 0.0010
Epoch 51/1000
1089/1089 [==============================] - 1s 973us/step - loss: 99291.7344 - mae: 148.2803 - val_loss: 116304.1875 - val_mae: 154.1186 - lr: 0.0010
Epoch 52/1000
1089/1089 [==============================] - 1s 977us/step - loss: 99103.1172 - mae: 147.6200 - val_loss: 116769.7500 - val_mae: 154.5076 - lr: 0.0010
Epoch 53/1000
1089/1089 [==============================] - 1s 972us/step - loss: 98979.3203 - mae: 147.3172 - val_loss: 110608.2344 - val_mae: 149.7165 - lr: 0.0010
Epoch 54/1000
1089/1089 [==============================] - 1s 972us/step - loss: 98993.3984 - mae: 147.1808 - val_loss: 113133.0625 - val_mae: 152.0357 - lr: 0.0010
Epoch 55/1000
1089/1089 [==============================] - 1s 973us/step - loss: 98810.0234 - mae: 147.2133 - val_loss: 118903.9297 - val_mae: 157.3274 - lr: 0.0010
Epoch 56/1000
1089/1089 

Epoch 100/1000
1089/1089 [==============================] - 1s 1ms/step - loss: 94346.4844 - mae: 143.6755 - val_loss: 119764.6172 - val_mae: 156.1121 - lr: 0.0010
Epoch 101/1000
1089/1089 [==============================] - 1s 998us/step - loss: 94530.9062 - mae: 143.3428 - val_loss: 115906.7500 - val_mae: 156.2811 - lr: 0.0010
Epoch 102/1000
1089/1089 [==============================] - 1s 986us/step - loss: 94491.2734 - mae: 143.6053 - val_loss: 115250.8359 - val_mae: 153.2468 - lr: 0.0010
Epoch 103/1000
1089/1089 [==============================] - 1s 985us/step - loss: 93872.4531 - mae: 142.5898 - val_loss: 114891.5000 - val_mae: 154.2710 - lr: 0.0010
Epoch 104/1000
1089/1089 [==============================] - 1s 993us/step - loss: 94325.4766 - mae: 142.9215 - val_loss: 111413.3828 - val_mae: 151.5546 - lr: 0.0010
Epoch 105/1000
1089/1089 [==============================] - 1s 994us/step - loss: 94053.9297 - mae: 143.0326 - val_loss: 112785.2031 - val_mae: 152.6826 - lr: 0.0010
Epoch 

1089/1089 [==============================] - 1s 976us/step - loss: 89924.3672 - mae: 139.3241 - val_loss: 113403.7031 - val_mae: 150.3303 - lr: 0.0010
Epoch 150/1000
1089/1089 [==============================] - 1s 986us/step - loss: 90253.7969 - mae: 139.7027 - val_loss: 117415.7344 - val_mae: 155.0459 - lr: 0.0010
Epoch 151/1000
1089/1089 [==============================] - 1s 974us/step - loss: 90084.6016 - mae: 139.4295 - val_loss: 115064.0938 - val_mae: 153.5325 - lr: 0.0010
Epoch 152/1000
1089/1089 [==============================] - 1s 977us/step - loss: 89949.9531 - mae: 139.2745 - val_loss: 115976.7500 - val_mae: 153.2559 - lr: 0.0010
Epoch 153/1000
1089/1089 [==============================] - 1s 974us/step - loss: 89886.8203 - mae: 139.2757 - val_loss: 115388.9609 - val_mae: 152.8247 - lr: 0.0010
Epoch 154/1000
1089/1089 [==============================] - 1s 1ms/step - loss: 86661.6328 - mae: 135.5797 - val_loss: 115221.5000 - val_mae: 151.8311 - lr: 1.0000e-04
Epoch 155/1000
10

1089/1089 [==============================] - 1s 1ms/step - loss: 84952.2109 - mae: 134.1184 - val_loss: 117575.1484 - val_mae: 153.1998 - lr: 1.0000e-04
Epoch 198/1000
1089/1089 [==============================] - 1s 990us/step - loss: 84919.3125 - mae: 134.0840 - val_loss: 117594.3281 - val_mae: 153.2823 - lr: 1.0000e-04
Epoch 199/1000
1089/1089 [==============================] - 1s 995us/step - loss: 84874.5625 - mae: 134.3660 - val_loss: 117663.4062 - val_mae: 152.3279 - lr: 1.0000e-04
Epoch 200/1000
1089/1089 [==============================] - 1s 998us/step - loss: 84866.3516 - mae: 134.0947 - val_loss: 118076.5000 - val_mae: 153.3569 - lr: 1.0000e-04
Epoch 201/1000
1089/1089 [==============================] - 1s 999us/step - loss: 84767.0547 - mae: 134.0033 - val_loss: 117847.3672 - val_mae: 153.4034 - lr: 1.0000e-04
Epoch 202/1000
1089/1089 [==============================] - 1s 1ms/step - loss: 84884.2500 - mae: 134.1994 - val_loss: 117219.7969 - val_mae: 152.5290 - lr: 1.0000e-04

Epoch 246/1000
1089/1089 [==============================] - 1s 1ms/step - loss: 83858.8594 - mae: 133.2957 - val_loss: 119057.5000 - val_mae: 153.9637 - lr: 1.0000e-04
Epoch 247/1000
1089/1089 [==============================] - 1s 1ms/step - loss: 83830.5156 - mae: 133.4184 - val_loss: 119636.1328 - val_mae: 154.1164 - lr: 1.0000e-04
Epoch 248/1000
1089/1089 [==============================] - 1s 1ms/step - loss: 83733.1797 - mae: 133.3977 - val_loss: 118673.7891 - val_mae: 152.9655 - lr: 1.0000e-04
Epoch 249/1000
1089/1089 [==============================] - 1s 1ms/step - loss: 83749.4375 - mae: 133.2712 - val_loss: 120053.7500 - val_mae: 154.0020 - lr: 1.0000e-04
Epoch 250/1000
1089/1089 [==============================] - 1s 1ms/step - loss: 83734.3203 - mae: 133.3357 - val_loss: 120185.8984 - val_mae: 154.8595 - lr: 1.0000e-04
Epoch 251/1000
1089/1089 [==============================] - 1s 1ms/step - loss: 83834.6875 - mae: 133.3761 - val_loss: 118730.9062 - val_mae: 154.1112 - lr: 1.0

In [39]:
# UPAC08 Predictions for all features

predict_upacs(model_dictionary=upac08_GhiGti, 
              upac_name='upac08',
              X_train=X_train, y_train=y_train,
              X_val=X_val, y_val=y_val,
              X_test=X_test, y_test=y_test)